In [1]:
import os
import sys
main_folder_path = os.path.abspath('..')
sys.path.append(main_folder_path)
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ogcc2w55
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ogcc2w55
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=e62e99caa55daabaf9440ca4199ca3bec6dd95e828244ac057697de9e27af8b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wlsmh4eq/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything


In [3]:
!git clone https://github.com/DWalicki95/MaterialsVision.git

Cloning into 'MaterialsVision'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 36 (delta 12), reused 15 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 263.49 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [4]:
%cd MaterialsVision

/content/MaterialsVision


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from materials_vision.data_loader import DataLoader

ModuleNotFoundError: No module named 'config'

In [ ]:
data = DataLoader().load_data_dict('AS')

In [ ]:
def show_random_img(data: dict, subseries_name: str, magnification=None, show:bool = True, random_seed=42):
    if random_seed > -1:
        np.random.seed(random_seed)
    images_paths = data[subseries_name]
    if magnification:
        images_paths = str(magnification) in images_paths
    rand_img = np.random.choice(images_paths)
    rand_img = cv2.imread(rand_img)
    if show:
        plt.imshow(rand_img)
        # plt.axis('off')
        print(rand_img.shape)
    return rand_img

In [ ]:
def crop_image(img, height_after_cropping: int = 960):
    return img[0:height_after_cropping]

In [ ]:
img = show_random_img(data, 'AS3', random_seed=-1)

In [ ]:
cropped_img = crop_image(img)
plt.imshow(cropped_img)

<H1> Automatically generating masks

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from materials_vision.config import SAM_VIT_B_WEIGHTS

In [ ]:
sam = sam_model_registry['vit_b'](checkpoint=SAM_VIT_B_WEIGHTS)
predictor = SamPredictor(sam)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=SAM_VIT_B_WEIGHTS)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)